In [1]:
#IMPORT DEPENDENCIES

import pandas as pd
import datetime as dt
import glob
import csv


    RIDE DATA

In [2]:
#GATHER URLS, WRITE TO CSV
urls = [#2019 Jan-May
        'https://s3.amazonaws.com/tripdata/JC-201905-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201904-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201903-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201902-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201901-citibike-tripdata.csv.zip',
        #2018 Jan-May
        'https://s3.amazonaws.com/tripdata/JC-201805-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201804-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201803-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201802-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201801-citibike-tripdata.csv.zip',
       ]
dfs = [pd.read_csv(url) for url in urls]

df = pd.concat(dfs, ignore_index=True, sort=True)


In [3]:
df.head()

,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype
0,29224,1988,3279,40.721630,-74.049968,Dixon Mills,1,3185,40.717733,-74.043845,City Hall,2019-05-01 00:07:36.4670,2019-05-01 00:12:47.6350,311,Subscriber
1,26170,1974,3203,40.727596,-74.044247,Hamilton Park,1,3681,40.715178,-74.037683,Grand St,2019-05-01 00:08:29.2840,2019-05-01 00:17:27.9410,538,Subscriber
2,29517,1988,3639,40.719252,-74.034234,Harborside,1,3638,40.724294,-74.035483,Washington St,2019-05-01 00:39:36.4750,2019-05-01 00:42:34.9640,178,Subscriber
3,29606,1986,3212,40.734786,-74.050444,Christ Hospital,2,3206,40.731169,-74.057574,Hilltop,2019-05-01 01:24:28.0220,2019-05-01 01:29:45.5730,317,Subscriber
4,26224,1986,3202,40.727224,-74.033759,Newport PATH,2,3639,40.719252,-74.034234,Harborside,2019-05-01 01:54:39.6660,2019-05-01 02:00:34.1750,354,Customer


In [4]:
#INDEX starttime column
def f(df):
    df['Year'] = pd.DatetimeIndex(df['starttime']).year
    df['Month_Name'] = pd.DatetimeIndex(df['starttime']).month_name()
    df['Month'] = pd.DatetimeIndex(df['starttime']).month
    df['Hour'] = pd.DatetimeIndex(df['starttime']).hour
    return df

In [5]:
df_index = f(df)
df_index.head()

,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype,Year,Month_Name,Month,Hour
0,29224,1988,3279,40.721630,-74.049968,Dixon Mills,1,3185,40.717733,-74.043845,City Hall,2019-05-01 00:07:36.4670,2019-05-01 00:12:47.6350,311,Subscriber,2019,May,5,0
1,26170,1974,3203,40.727596,-74.044247,Hamilton Park,1,3681,40.715178,-74.037683,Grand St,2019-05-01 00:08:29.2840,2019-05-01 00:17:27.9410,538,Subscriber,2019,May,5,0
2,29517,1988,3639,40.719252,-74.034234,Harborside,1,3638,40.724294,-74.035483,Washington St,2019-05-01 00:39:36.4750,2019-05-01 00:42:34.9640,178,Subscriber,2019,May,5,0
3,29606,1986,3212,40.734786,-74.050444,Christ Hospital,2,3206,40.731169,-74.057574,Hilltop,2019-05-01 01:24:28.0220,2019-05-01 01:29:45.5730,317,Subscriber,2019,May,5,1
4,26224,1986,3202,40.727224,-74.033759,Newport PATH,2,3639,40.719252,-74.034234,Harborside,2019-05-01 01:54:39.6660,2019-05-01 02:00:34.1750,354,Customer,2019,May,5,1


In [6]:
current_year = dt.datetime.now().year #get current year
ride_data = df_index.groupby(['Year', 'Month', 'Month_Name', 'Hour', 'birth year', 'gender', 'usertype'])\
                .agg({'bikeid': 'count', 'tripduration': 'mean'})\
                .reset_index()\
                .rename(columns={'bikeid': 'total_rides', 'tripduration':'avg_trip_duration'})
ride_data['age'] = (current_year - ride_data['birth year'])

ride_data.head()

,Year,Month,Month_Name,Hour,birth year,gender,usertype,total_rides,avg_trip_duration,age
0,2018,1,January,0,1957,1,Subscriber,4,78.750000,62
1,2018,1,January,0,1958,1,Subscriber,1,374.000000,61
2,2018,1,January,0,1962,1,Subscriber,13,695.384615,57
3,2018,1,January,0,1964,1,Subscriber,2,362.500000,55
4,2018,1,January,0,1968,1,Subscriber,1,2421.000000,51


In [7]:
ride_data.loc[(ride_data['gender']==1, 'gender')] = 'Male'
ride_data.loc[(ride_data['gender']==2, 'gender')] = 'Female'
ride_data.loc[(ride_data['gender']==0, 'gender')] = 'Unknown'
ride_data.head()

,Year,Month,Month_Name,Hour,birth year,gender,usertype,total_rides,avg_trip_duration,age
0,2018,1,January,0,1957,Male,Subscriber,4,78.750000,62
1,2018,1,January,0,1958,Male,Subscriber,1,374.000000,61
2,2018,1,January,0,1962,Male,Subscriber,13,695.384615,57
3,2018,1,January,0,1964,Male,Subscriber,2,362.500000,55
4,2018,1,January,0,1968,Male,Subscriber,1,2421.000000,51


In [8]:
ride_data.to_csv('ride_data_agg.csv', encoding='utf-8', index=False, )

TRIP DATA 

In [9]:
#GATHER URLS, WRITE TO CSV
urls = [ #2019
        'https://s3.amazonaws.com/tripdata/JC-201905-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201904-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201903-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201902-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201901-citibike-tripdata.csv.zip',
        #2018
        'https://s3.amazonaws.com/tripdata/JC-201812-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201811-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201810-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201809-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201808-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201807-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201806-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201805-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201804-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201803-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201802-citibike-tripdata.csv.zip',
        'https://s3.amazonaws.com/tripdata/JC-201801-citibike-tripdata.csv.zip',
       ]
dfs2 = [pd.read_csv(url) for url in urls]

df2 = pd.concat(dfs2, ignore_index=True, sort=True)

df2.head()

,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype
0,29224,1988,3279,40.721630,-74.049968,Dixon Mills,1,3185,40.717733,-74.043845,City Hall,2019-05-01 00:07:36.4670,2019-05-01 00:12:47.6350,311,Subscriber
1,26170,1974,3203,40.727596,-74.044247,Hamilton Park,1,3681,40.715178,-74.037683,Grand St,2019-05-01 00:08:29.2840,2019-05-01 00:17:27.9410,538,Subscriber
2,29517,1988,3639,40.719252,-74.034234,Harborside,1,3638,40.724294,-74.035483,Washington St,2019-05-01 00:39:36.4750,2019-05-01 00:42:34.9640,178,Subscriber
3,29606,1986,3212,40.734786,-74.050444,Christ Hospital,2,3206,40.731169,-74.057574,Hilltop,2019-05-01 01:24:28.0220,2019-05-01 01:29:45.5730,317,Subscriber
4,26224,1986,3202,40.727224,-74.033759,Newport PATH,2,3639,40.719252,-74.034234,Harborside,2019-05-01 01:54:39.6660,2019-05-01 02:00:34.1750,354,Customer


In [10]:
#INDEX starttime column
def f2(df2):
    df2['SYear'] = pd.DatetimeIndex(df2['starttime']).year
    df2['SMonth_Name'] = pd.DatetimeIndex(df2['starttime']).month_name()
    df2['SMonth'] = pd.DatetimeIndex(df2['starttime']).month
    df2['SHour'] = pd.DatetimeIndex(df2['starttime']).hour
    df2['EHour'] = pd.DatetimeIndex(df2['stoptime']).hour
    return df2

In [11]:
df_index2 = f2(df2)
df_index2.head()

,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype,SYear,SMonth_Name,SMonth,SHour,EHour
0,29224,1988,3279,40.721630,-74.049968,Dixon Mills,1,3185,40.717733,-74.043845,City Hall,2019-05-01 00:07:36.4670,2019-05-01 00:12:47.6350,311,Subscriber,2019,May,5,0,0
1,26170,1974,3203,40.727596,-74.044247,Hamilton Park,1,3681,40.715178,-74.037683,Grand St,2019-05-01 00:08:29.2840,2019-05-01 00:17:27.9410,538,Subscriber,2019,May,5,0,0
2,29517,1988,3639,40.719252,-74.034234,Harborside,1,3638,40.724294,-74.035483,Washington St,2019-05-01 00:39:36.4750,2019-05-01 00:42:34.9640,178,Subscriber,2019,May,5,0,0
3,29606,1986,3212,40.734786,-74.050444,Christ Hospital,2,3206,40.731169,-74.057574,Hilltop,2019-05-01 01:24:28.0220,2019-05-01 01:29:45.5730,317,Subscriber,2019,May,5,1,1
4,26224,1986,3202,40.727224,-74.033759,Newport PATH,2,3639,40.719252,-74.034234,Harborside,2019-05-01 01:54:39.6660,2019-05-01 02:00:34.1750,354,Customer,2019,May,5,1,2


In [12]:
trip_data = df_index2.groupby(['SYear', 'SMonth', 'SMonth_Name', 'SHour', 'EHour',\
                              'start station name','start station latitude', 'start station longitude',\
                              'end station name', 'end station latitude', 'end station longitude'])\
                .agg({'bikeid': 'count', 'tripduration': 'mean'})\
                .reset_index()\
                .rename(columns={'bikeid': 'total_rides', 'tripduration':'avg_trip_duration'})

trip_data.head()

,SYear,SMonth,SMonth_Name,SHour,EHour,start station name,start station latitude,start station longitude,end station name,end station latitude,end station longitude,total_rides,avg_trip_duration
0,2018,1,January,0,0,5 Corners Library,40.734961,-74.059503,Christ Hospital,40.734786,-74.050444,1,290.0
1,2018,1,January,0,0,Baldwin at Montgomery,40.723659,-74.064194,West Side Light Rail,40.714402,-74.088772,1,741.0
2,2018,1,January,0,0,Brunswick St,40.724176,-74.050656,Baldwin at Montgomery,40.723659,-74.064194,1,486.0
3,2018,1,January,0,0,Brunswick St,40.724176,-74.050656,Hilltop,40.731169,-74.057574,1,312.0
4,2018,1,January,0,0,Christ Hospital,40.734786,-74.050444,Hilltop,40.731169,-74.057574,1,305.0


In [13]:
trip_data.to_csv('tripdata_2018_May_2019.csv', encoding='utf-8', index=False, )